Equatorial Slices
============

**Summary:**    2-D profiles of selected output variables in the equatorial plane with each output quantity $q_i$ defined such that

$q_i(r,\phi) = f_i(r,\theta=\pi/2,\phi)$

for each specified output $f_i$.

**Subdirectory:**  Equatorial_Slices

**main_input prefix:** equatorial

**Python Class:** Equatorial_Slices

**Additional Namelist Variables:**  
None

The equatorial-slice output type allows us to examine how the fluid properties vary in longitude and radius.

Examining the *main_input* file, we see that the following output values have been denoted for the Equatorial Slices (see *rayleigh_output_variables.pdf* for mathematical formulae):


| Menu Code  | Description |
|------------|-------------|
| 1          | Radial Velocity |
| 2          | Theta Velocity |
| 3          | Phi Velocity  |




In the example that follows, we demonstrate how to create a 2-D plot of radial velocity in the equatorial plane (at a single time step).

We begin with the usual preamble.  Examining the data structure, we see that the *vals* array is dimensioned to account for longitudinal variation, and that we have the new coordinate attribute *phi*.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

model_type = 2 # 1 for anelastic example, 2 Boussinesq example
base_dir = '/home/nfeatherstone/runs/cig_vis/'
font_size=14     # Font size for plot labels 

if (model_type == 1):
    model_dir = base_dir+'anelastic/'
    # Define some units for plotting purposes
    eunits = r'(erg cm$^{-3}$)'                # energy density
    tunits = '(s)'                             # time
    vunits = r'(cm s$^{-1}$)'                  # velocity
    dunits = '(cm)'                            # distance
    thermal_label = r' Specific Entropy '      # specific entropy
    thermal_units = r'(erg g$^{-1}$ K$^{-1}$)' # specific entropy
    lunits = r'(erg s$^{-1}$)'                 # energy / time (luminosity)
    funits = '(nHz)'                           # Frequency units
    mfunits = '(g cm$^{-2}$ s$^{-1}$)'
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series
    
if (model_type == 2):
    model_dir = base_dir+'Boussinesq/'
    # Define some units for plotting purposes
    eunits = ''                          # energy density
    tunits = '(viscous diffusion times)' # time
    vunits = ''                          # velocity
    dunits = ''                          # distance
    thermal_label = ' Temperature ' 
    thermal_units = '' 
    lunits = ''                          # energy / time (luminosity)
    funits = ''                          # Frequency units
    mfunits = ''
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series

In [ ]:
from rayleigh_diagnostics import Equatorial_Slices, build_file_list
import numpy
import matplotlib.pyplot as plt
from matplotlib import ticker, font_manager
files = build_file_list(imin,imax,path=model_dir+'Equatorial_Slices')
nf = len(files)
es = Equatorial_Slices(files[nf-1],path='')
help(es)

In [ ]:
################################
# Equatorial Slice 

tindex =1 # Grab second time index from this file
qindex = es.lut[1] # radial velocity

#Set up the grid
remove_mean = True # Remove the m=0 mean
nr = es.nr
nphi = es.nphi
r = es.radius/numpy.max(es.radius)
phi = numpy.zeros(nphi+1,dtype='float64')
phi[0:nphi] = es.phi
phi[nphi] = numpy.pi*2  # For display purposes, it is best to have a redunant data point at 0,2pi

#We need to generate a cartesian grid of x-y coordinates (both X & Y are 2-D)
radius_matrix, phi_matrix = numpy.meshgrid(r,phi)
X = radius_matrix * numpy.cos(phi_matrix)
Y = radius_matrix * numpy.sin(phi_matrix)

field = numpy.zeros((nphi+1,nr),dtype='float64')
field[0:nphi,:] =es.vals[:,:,qindex,tindex]
field[nphi,:] = field[0,:]  #replicate phi=0 values at phi=2pi

#remove the mean if desired (usually a good idea, but not always)
if (remove_mean):
    for i in range(nr):
        the_mean = numpy.mean(field[:,i])
        field[:,i] = field[:,i]-the_mean

#Plot
sizetuple=(8,5)
fig, ax = plt.subplots(figsize=(8,8))
plt.rcParams.update({'font.size': font_size})
tsize = 20     # title font size
cbfsize = 10   # colorbar font size
img = ax.pcolormesh(X,Y,field,cmap='RdYlBu_r',shading='auto')
ax.axis('equal')  # Ensure that x & y axis ranges have a 1:1 aspect ratio
ax.axis('off')    # Do not plot x & y axes

# Plot bounding circles
ax.plot(r[nr-1]*numpy.cos(phi), r[nr-1]*numpy.sin(phi), color='black')  # Inner circle
ax.plot(r[0]*numpy.cos(phi), r[0]*numpy.sin(phi), color='black')  # Outer circle

ax.set_title(r'$v_r$', fontsize=20)

#colorbar ...
cbar = plt.colorbar(img,orientation='horizontal', shrink=0.5, aspect = 15, ax=ax)
cbar.set_label(vunits)
        
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()
cbar.ax.tick_params(labelsize=cbfsize)   #font size for the ticks

t = cbar.ax.xaxis.label
t.set_fontsize(cbfsize)  # font size for the axis title


plt.tight_layout()
plt.show()